In [10]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox,BRepPrimAPI_MakeCylinder
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut, BRepAlgoAPI_Defeaturing
from OCC.Core.BRepFilletAPI import BRepFilletAPI_MakeFillet
from OCC.Core.gp import gp_Vec
from OCC.Extend.ShapeFactory import translate_shp
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopTools import TopTools_ListOfShape
from OCC.Extend.TopologyUtils import TopologyExplorer

In [49]:
box_shape = BRepPrimAPI_MakeBox(20, 20, 5).Shape()

fillet = BRepFilletAPI_MakeFillet(box_shape)
# Add fillet on one edge
for i,e in enumerate(TopologyExplorer(box_shape).edges()):
    if i == 5:
        fillet.Add(2, e)
        break
blended_box = fillet.Shape()

cylinder_shape = translate_shp(BRepPrimAPI_MakeCylinder(3, 5).Shape(), gp_Vec(10, 10, 0))
cut_shape = BRepAlgoAPI_Cut(blended_box, cylinder_shape).Shape()

In [50]:
def defeature_faces_from_solid(aSolid, aFaces):
    '''
    deafeature faces from a solid
    @param aSolid:   the Topo_Solid to remove the faces from
    @param aFaces:   the list of Topo_Face to remove
    '''
    
    occ_seq = TopTools_ListOfShape()
    for aFace in aFaces:
        occ_seq.Append(aFace)
    aDF = BRepAlgoAPI_Defeaturing()          # Defeaturing algorithm
    aDF.SetShape(aSolid)                     # Set the shape
    aDF.AddFacesToRemove(occ_seq)            # Add faces to remove
    aDF.SetRunParallel(False)                # Define the processing mode (parallel or single)
    aDF.SetToFillHistory(False)              # Define whether to track the shapes modifications
    aDF.Build()
    
    if aDF.IsDone():
        print("defeaturing is done")
    return aDF.Shape()

In [54]:
my_renderer = JupyterRenderer()
my_renderer.DisplayShape(cut_shape, render_edges=True, topo_level="Face", shape_color="#b2d1ba", update=True)

In [58]:
sel_shape = my_renderer.GetSelectedShape()
def_shape = defeature_faces_from_solid(cut_shape, [sel_shape])

defeaturing is done


In [59]:
my_renderer_2 = JupyterRenderer()
my_renderer_2.DisplayShape(def_shape, render_edges=True, topo_level="Face", shape_color="#b2d1ba", update=True)